<a href="https://colab.research.google.com/github/jumafernandez/UNLP/blob/master/TFI/features_estaticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Descargo el archivo con las consultas que está en Github
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/TFI/data/Correos_Seleccionados_y_Etiquetados.csv

# Leo el archivo en un dataframe
import pandas as pd

df = pd.read_csv('Correos_Seleccionados_y_Etiquetados.csv', delimiter="|")
df.info()

# Vemos la cantidad de Instancias por clase
df.Clase.value_counts()

# Transformamos todas las Clases minoritarias (Puedo ir variando la cantidad de clases que derivo a la Clase "Otras Consultas")
cantidad_clases=3
clases = df.Clase.value_counts()
clases_minoritarias = clases.iloc[cantidad_clases:].keys().to_list()
df.Clase[df['Clase'].isin(clases_minoritarias)] = "Otras Consultas"

df.Clase.value_counts()


--2020-12-08 02:33:49--  https://raw.githubusercontent.com/jumafernandez/UNLP/master/TFI/data/Correos_Seleccionados_y_Etiquetados.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448336 (438K) [text/plain]
Saving to: ‘Correos_Seleccionados_y_Etiquetados.csv.1’

Correos_Seleccionad 100%[===================>] 437.83K  --.-KB/s    in 0.03s   

2020-12-08 02:33:49 (13.7 MB/s) - ‘Correos_Seleccionados_y_Etiquetados.csv.1’ saved [448336/448336]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Fecha              1000 non-null   object
 1   Hora               1000 non-null   object
 2   Apellido y Nombre  1000 non

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Otras Consultas             399
Boleto Universitario        240
Ingreso a la Universidad    232
Requisitos de Ingreso       129
Name: Clase, dtype: int64

In [7]:
df.shape

(1000, 11)

In [33]:
# Features estáticas léxicas: basadas en caracteres

# Total number of characters
def cantidad_caracteres(columna_consulta):
  return columna_consulta.str.len()

df['cantidad_caracteres'] = cantidad_caracteres(df.Consulta)

# Proporcion de mayúsculas en la consulta
def proporcion_mayusculas(consulta):
  return sum(1 for letra in consulta if letra.isupper())

df['proporcion_mayusculas'] = df['Consulta'].apply(proporcion_mayusculas)/df['cantidad_caracteres']

# Proporción de letras en la consulta
def proporcion_letras(consulta):
  return sum(1 for letra in consulta if letra.isalpha())

df['proporcion_letras'] = df['Consulta'].apply(proporcion_letras)/df['cantidad_caracteres']

# Cantidad de letras con tildes
def cantidad_tildes(consulta):
  return sum(1 for letra in consulta if letra.lower() in ['á', 'é', 'í', 'ó', 'ú'])

df['cantidad_tildes'] = df['Consulta'].apply(cantidad_tildes)


0      hola quiero anotarme a las materias ,para el s...
1      hola buenos días! quería saber cuando voy a po...
2      hola quisiera saber si en la consulta de situa...
3      buenas noches. en mi situacion academica apare...
4      hola,  quisiera obtener mi promedio o saber co...
                             ...                        
995    buenas tardes. a quién debo contactar para que...
996    hola queria saber porque no me publicaron la n...
997    hola buenas noches quería saber alguna fecha e...
998    hola quisiera saber que pasa con los examenes ...
999            necesitaria saber mi aituacion academica 
Name: Consulta, Length: 1000, dtype: object

In [47]:
# Features estáticas léxicas (palabras)

# total number of words
def cantidad_palabras(consulta):
  palabras = consulta.split(sep=' ')
  return len(palabras)

df['cantidad_palabras'] = df['Consulta'].apply(cantidad_palabras)

# proportion of short words (less than four characters)
def cantidad_palabras_cortas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return sum(1 for palabra in palabras if len(palabra) <= letras_corta)

df['cantidad_palabras_cortas'] = df['Consulta'].apply(cantidad_palabras_cortas)

# ratio of number of distinct words to the total number of words: |set(words)|/|words|
def cantidad_palabras_distintas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return len(set(palabras))

df['proporcion_palabras_distintas'] = df['Consulta'].apply(cantidad_palabras_distintas)/df['cantidad_palabras']
df['proporcion_palabras_distintas']


0      0.857143
1      0.916667
2      0.794118
3      0.969697
4      0.866667
         ...   
995    0.958333
996    0.925926
997    0.857143
998    0.920000
999    1.000000
Name: proporcion_palabras_distintas, Length: 1000, dtype: float64

In [48]:
# Features estáticas sintácticas

# Frecuencia de signos de puntuación {,.¿?!:;’"}
def cantidad_signos_puntuacion(consulta):
  signos_puntuacion = [',', '.', '¿', '?', '!', '¡', ':', ';', '"']
  return sum(1 for letra in consulta if letra.lower() in signos_puntuacion)

df['frecuencia_signos_puntuacion'] = df['Consulta'].apply(cantidad_signos_puntuacion)/df['cantidad_caracteres']


0      0.030864
1      0.018315
2      0.005556
3      0.027027
4      0.038462
         ...   
995    0.030769
996    0.008000
997    0.008621
998    0.000000
999    0.000000
Name: frecuencia_signos_puntuacion, Length: 1000, dtype: float64

In [76]:
# Features estáticas estructurales

# total number of sentences
def cantidad_oraciones(consulta):
  if consulta.count('.')==0:
    return 1
  else: 
    return consulta.count('.')

df['cantidad_oraciones'] = df['Consulta'].apply(cantidad_oraciones)

# utiliza código de asignatura?
def utiliza_codigo_asignatura(consulta):
  codigo=0
  palabras = consulta.split(sep=' ')
  for palabra in palabras:
    if palabra.isdigit():
      if int(palabra)>=10000 and int(palabra)<=99999:
        codigo=1
  return codigo

df['utiliza_codigo_asignatura'] = df['Consulta'].apply(utiliza_codigo_asignatura)

# Tiene legajo?
def posee_valor(atributo):
  return 1-atributo.isna()
  
df['posee_legajo'] = posee_valor(df['Legajo'])

# Tiene teléfono? 
df['posee_telefono'] = posee_valor(df['Teléfono'])

0      1
1      3
2      1
3      5
4      2
      ..
995    2
996    1
997    1
998    1
999    1
Name: cantidad_oraciones, Length: 1000, dtype: int64

In [85]:
# Features estáticas de dominio específico

def servicio_email(consulta, proveedor):  
  if consulta.lower().find(proveedor)==-1:
    return 0
  else:
    return 1
# Correo Gmail?
df['correo_gmail'] = df['E-mail'].apply(servicio_email, proveedor='gmail')

# Correo yahoo?
df['correo_yahoo'] = df['E-mail'].apply(servicio_email, proveedor='yahoo')

# Correo hotmail?
df['correo_hotmail'] = df['E-mail'].apply(servicio_email, proveedor='hotmail')


417